In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from classification import train_classifier, classify, read_csv, parse

train_dir = '00_input/train'
im_size = 200
classes = 50

/home/g5sokolovroman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Model
from keras import backend as K
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence

from keras.layers import (Input, concatenate, Conv2D, MaxPooling2D, 
                          UpSampling2D, Convolution2D, ZeroPadding2D, 
                          BatchNormalization, Activation, concatenate, 
                          Flatten, Dense, merge, Dropout)
from keras.optimizers import Adam
from keras.applications import resnet50, vgg16, vgg19, inception_v3


def get_model():
    base_model = resnet50.ResNet50(
        weights='imagenet', 
        include_top=False, 
        input_shape=(im_size, im_size, 3))
    
    base_model.trainable = False
    
    base_out = base_model.get_layer('add_8').output
    
    maxpool = MaxPooling2D()(base_out)
    
    #dropout = Dropout()(maxpool)
    
    #conv = Conv2D(filters=512, kernel_size=(3,3), padding='same')(maxpool)
    #relu = Activation('relu')(conv)
    #batchnorm = BatchNormalization()(relu)
    #maxpool = MaxPooling2D()(batchnorm)
    
    #dropout = Dropout(0.2)(maxpool)
    
    #conv = Conv2D(filters=512, kernel_size=(3,3), padding='same')(dropout)
    #relu = Activation('relu')(conv)
    #batchnorm = BatchNormalization()(relu)
    #maxpool = MaxPooling2D()(batchnorm)
    
    flatten = Flatten()(maxpool)
    dense = Dense(100, activation='relu')(flatten)
    dense = Dense(classes, activation='softmax')(dense)
    
    #for layer in base_model.layers:
    #    if layer.name[:3] == 'con' or layer.name[:3] == 'res':
    #        layer.trainable = False
    #    layer.trainable = False
    #    if layer.name == 'add_3':
    #        break
            
    
    inputs = base_model.input
    model = Model(inputs=inputs, outputs=dense)
    model.compile(
        optimizer=Adam(0.0005, decay=0), 
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [3]:
model_name = 'exp_{epoch:d}_{val_acc:.2f}.hdf5'
train_X = np.load('train_X.npy')
train_Y = np.load('train_Y.npy')
model = get_model()
model.summary()
checkpoint = ModelCheckpoint(
    model_name, 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    period=1,
    save_weights_only=False)
try:
    model.fit(train_X, train_Y, epochs=100, batch_size=100, callbacks=[checkpoint], validation_split=(1/6))
except KeyboardInterrupt:
    print('\nTraining interrupted')

/home/g5sokolovroman/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Train on 2083 samples, validate on 417 samples
Epoch 1/100
2083/2083 [==============================] - 44s 21ms/step - loss: 4.5542 - acc: 0.0269 - val_loss: 3.9062 - val_acc: 0.0456

Epoch 00001: val_acc improved from -inf to 0.04556, saving model to exp_1_0.05.hdf5
Epoch 2/100
2083/2083 [==============================] - 33s 16ms/step - loss: 3.8417 - acc: 0.0341 - val_loss: 3.9368 - val_acc: 0.0288

Epoch 00002: val_acc did not improve from 0.04556
Epoch 3/100
1300/2083 [=================>............] - ETA: 11s - loss: 3.7765 - acc: 0.0277
Training interrupted


In [ ]:
train_gt = read_csv(train_dir+'/gt.csv')
train_X, train_Y = parse(train_gt, train_dir+'/images', True, False)
np.save('train_X.npy', train_X)
np.save('train_Y.npy', train_Y)

In [3]:
model = get_model()
model.summary()

/home/g5sokolovroman/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [3]:
base_model = resnet50.ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(im_size, im_size, 3))
base_model.summary()

/home/g5sokolovroman/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [4]:
from keras.applications import resnet50, vgg16, vgg19, inception_v3
base_model = inception_v3.InceptionV3(
    weights='imagenet', 
    include_top=False, 
    input_shape=(im_size, im_size, 3))
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 99, 99, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 99, 99, 32)   96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 99, 99, 32)   0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 